In [267]:
import pandas as pd
data_all = pd.read_csv('/users/chineduezigbo/desktop/movies.csv')
data = data_all.loc[data_all['country'] == 'United States'] #To just select the movies from United States
data.dropna(inplace=True) #Drop all null values in any column of the dataset

import altair as alt
from altair.expr import datum
alt.data_transformers.enable('default', max_rows=8000) #altair supports only 5000 records, so added this to make it work for the taken dataset

splom_brush = alt.selection(type="interval", encodings=['x']) #interval brush used in SPLOM charts it is encoded with respect to x-axis
legend_selection = alt.selection_multi(fields=['rating'], bind='legend') #Legend selection in SPLOM charts based on Rating field 
genre_selection = alt.selection(type="single", encodings=['x']) #Genre selection in chart 2 which is passed onto the chart 3 and SPLOM charts 
release_brush = alt.selection(type='interval', encodings=['x']) #Release date range selection in chart 1 which is passed onto all below charts


#Creating a base chart with release date and gross chart on top of which a small chart will be encoded
base = alt.Chart(data).mark_bar(size=10).encode( #The bar width is set to size 10
    x = 'Released on:T', #Release date is on x-axis of chart 1
    y = 'gross:Q', #Gross amount date is on y-axis of chart 1
).properties( #Property block controls the height and width of the chart
    width=600, 
    height=400
)

#This chart uses the same parameters of base chart but the size of it is small to fit the other charts in the screen
lower = base.properties(
    height=60
).add_selection(release_brush)

release_chart = lower #assigning the lower chart above to a variable to concat at the end

#This chart shows all Genre with the average votes for each Genre
chart = alt.Chart(data).mark_bar().encode( 
    y=alt.Y('mean(votes):Q'), #Y-axis is the average/mean of votes
    x=alt.X("genre:N",sort='-y'), #X-axis is the all Genre
    color=alt.condition(genre_selection, alt.ColorValue("steelblue"), alt.ColorValue("lightgrey")) #Color condition is used for highlighting the selected bar in blue and other bar in gray
).add_selection(genre_selection #Add Selection block contains all the filters from other charts -> genre_selection is from this chart, Splom brush from SPLOM chart selection, legend selection filter is also from SPLOM chart, release brush is from chart 1
                ).transform_filter(splom_brush).transform_filter(legend_selection).transform_filter(release_brush) 


ranked_text = alt.Chart(data).mark_text(align='right').encode( #This is the chart where the data resides at the back of chart 2 to make sure that the selected bar movies are passed to next charts
    y=alt.Y('row_number:O',axis=None) #Row number of the data will be Y axis to store the selected data to pass it on
).transform_filter(
    genre_selection  #release brush from chart 2
).transform_filter(splom_brush).transform_filter(legend_selection).transform_window(  #trnaform window helps to perform required calculations over selected groups of data. 
    row_number='row_number()'
).transform_filter(release_brush  #release brush from chart 1
).transform_filter(
    'datum.row_number < 20' #Only top20 movies by gross is displayed considering the screen space
) #both filters from SPLOM chart

chart2_bar = ranked_text.mark_bar(width=20).encode( #above selected data from ranked_text is used as base of this chart
    y=alt.Y('name:N',sort='-x'), #Y axis is the name of the movie and it is sorted based on gross in descending order
    x='gross:Q', #x axis is the gross values of the movies
    tooltip = ['name:N','Year:N','score:Q','votes:Q','director:N','star:N','company:N','budget:Q','gross:Q','runtime:Q','rating:N','genre:N'], #mouseover/tooltip is added for better user experience
    order=alt.Order("gross:Q"), #ordering the data based on the gross
).transform_filter(splom_brush).transform_filter(legend_selection).transform_filter(genre_selection #filters from all other charts for linked highlightning
).transform_filter(release_brush)


SPLOM = alt.Chart(data).mark_circle().encode( #SPLOM charts is built with circle chart
    alt.X(alt.repeat("column"), type='quantitative'), #X-axis takes values from column entered below
    alt.Y(alt.repeat("row"), type='quantitative'), #Y-axis takes values from rows entered below
    tooltip = ['name:N','score:Q','Year:N','votes:Q','director:N','star:N','company:N','budget:Q','gross:Q','runtime:Q','rating:N','genre:N'], #mouseover/tooltip is added for better user experience
    color='rating:N' #Splom chart is colored based on ratings which is also used in legend selection
).properties(  #Property block controls the height and width of the chart
    width=150,
    height=150
).repeat(  #repeat block take care of the SPLOM design with the rows and columns selected
    row=['budget', 'runtime', 'votes','gross'],
    column=['gross','votes','runtime','budget'] #columns field order is to be reverse order of the rows
).add_selection(splom_brush,legend_selection #2 different selections are available in SPLOM charts - Brush selection with respect to x-axis and legend selection based on rating
                ).transform_filter(genre_selection 
).transform_filter(splom_brush).transform_filter(legend_selection).transform_filter(release_brush).interactive() #This chart is interactive


Row_1 = alt.hconcat( #Horizontally concatenating Chart 2 and Chart 3 to place it on Row 2
    chart,
    chart2_bar)

Row_2 = release_chart & Row_1 & SPLOM #All charts are vertically concatenated
Row_2.resolve_scale(
    color='independent' #This make sure that the legends are separate for all charts
).configure_view(strokeWidth=0)

/var/folders/1g/3t64p0c933987f0wkv334_9m0000gn/T/ipykernel_14867/1574642405.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True) #Drop all null values in any column of the dataset
/opt/homebrew/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)